# Web-scraping ресторанов с tripadvisor 

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from time import time, sleep
from random import randint
from IPython.core.display import clear_output


session = requests.session() # Создается сессия
session.headers.update ({'User-Agent': "Mozilla/5.0 (X11; Linux x86_64; rv:46.0) Gecko/20100101 Firefox/46.0",
                  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                  'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
                  'Connection': 'keep-alive'})

names, ratings, costs, reviews, cuisines = [], [], [], [], [] # Списки для хранения полученных данных

# Подготовка к мониторингу цикла
start_time = time()
requests = 0

# Для каждой страницы в интервале
for offset in range (0, 961, 30):

    # Создать GET запрос
    response = session.get('https://www.tripadvisor.ru/Restaurants-g298484-oa{}-Moscow_Central_Russia.html#EATERY_OVERVIEW_BOX'.format (offset))
    
    # Задать паузу
    sleep(randint(5, 8))
    
    # Отслеживание запросов, чтобы не нагружать сервер
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output (wait=True)
    
    # Выбросить предупреждение, если статус не 200
    if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))
            
    # Парсим содержимое запроса, используя BeautifuSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Выбираем все 30 контейнеров-ресторанов со страницы
    rests = soup.select('div .listing.rebrand')
    
    # Словарь для перевода стоимости из символов в количество рублей
    real_costs = {'$': 700, '$$ - $$$': 1500,'$$$$': 3000}

    # Извлечь нужные данные из каждого контейнера
    for rest in rests:
    
        # Необходимое условие - наличие у ресторана отзывов и стоимости среднего чека
        if rest.find('span', 'reviewCount') is not None and rest.find('span', 'item price') is not None:
        
            # Название ресторана
            name = rest.find('div', 'title').a.text
            names.append(name)
        
            # Рейтинг ресторана, основанный на отзывах посетителей
            rating = rest.find('span', 'ui_bubble_rating')['alt']
            ratings.append(rating)
        
            # Средний чек в ресторане
            cost = rest.find('span', 'item price').text
            # Перевод в рубли и добавление в список costs
            costs.append(real_costs[cost])
        
            # Основные направления кухни в данном ресторане
            cuisine = ','.join(map(lambda x: x.text, rest.find_all(['span','a'], 'item cuisine')))
            cuisines.append(cuisine)
        
            # Количество отзывов
            review = rest.find('span', 'reviewCount').a.text
            # У некоторых строк имеется символ прерывания, например в строке "1 256 отзывов" между 1 и 256, 
            # который усложняет дальнейший анализ
            # Обработаем такую строку и добавим в список reviews 
            reviews.append(re.sub('\s+', '', review))
            
rests_df = pd.DataFrame({'restaurant': names,
                    "user's rating": ratings,
                    'average check': costs,
                    'cuisines': cuisines,
                    'number of reviews': reviews})

# Для удобства сменим порядок столбцов
rests_df = rests_df[['restaurant',  'cuisines', 'average check', 'number of reviews', "user's rating"]]
# Названия ресторанов. Удалим лишние пробелы
rests_df['restaurant'] = rests_df['restaurant'].str.strip()
# Отзывы. Извлекаем только цифры, переводим из str в int
rests_df['number of reviews'] = rests_df['number of reviews'].str.extract('(\d+)').astype(int)
# Рейтинг. Убираем лишнию строку, меняем знак и приводим к типу float
rests_df["user's rating"] = rests_df["user's rating"].str.replace(' из 5 кружков', '')
rests_df["user's rating"] = rests_df["user's rating"].str.replace(',', '.').astype(float)

# Сохраняем в формате csv
rests_df.to_csv('rests.csv')